# FASTQ Processing

> Processing FASTQ files for barcode extraction and classification

## Introduction
This notebook implements the FASTQ processing functionality for BarcodeSeqKit. It includes functions for handling FASTQ files, extracting barcodes, and sorting reads into appropriate output files based on barcode detection.


In [ ]:
#| default_exp fastq_processing

In [ ]:
#| hide
from nbdev.showdoc import *


In [ ]:
#| export
import os
import re
import gzip
import logging
import shutil
from typing import List, Dict, Tuple, Set, Optional, Union, Iterator, Any
from pathlib import Path
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import fnmatch
from tqdm.auto import tqdm

from BarcodeSeqKit.core import (
    BarcodeConfig, 
    BarcodeMatch, 
    OrientationType,
    ExtractionStatistics,
    BarcodeLocationType,
    BarcodeExtractorConfig
)
from BarcodeSeqKit.sequence_utils import classify_read_by_first_match

## FASTQ File Handler

In [ ]:
#| export
class FastqHandler:
    """Handles FASTQ file operations."""
    
    @staticmethod
    def find_fastq_pairs(directory: str) -> Tuple[str, str]:
        """Find paired FASTQ files in a directory.
        
        Args:
            directory: Directory to search for FASTQ files
            
        Returns:
            Tuple of (read1_path, read2_path)
            
        Raises:
            ValueError: If paired FASTQ files cannot be found
        """
        files = os.listdir(directory)
        # Common naming patterns for paired FASTQ files
        r1_patterns = ('_1.fq.gz', '_R1.fastq.gz', '_R1_001.fastq.gz')
        r2_patterns = ('_2.fq.gz', '_R2.fastq.gz', '_R2_001.fastq.gz')
        
        # Find files matching the patterns
        r1_file = None
        r2_file = None
        
        for f in files:
            if any(f.endswith(pattern) for pattern in r1_patterns):
                r1_file = f
            elif any(f.endswith(pattern) for pattern in r2_patterns):
                r2_file = f
        
        if not (r1_file and r2_file):
            raise ValueError(f"Could not find paired FASTQ files in {directory}")
            
        return os.path.join(directory, r1_file), os.path.join(directory, r2_file)
    
    @staticmethod
    def open_fastq(filepath: str) -> Union[gzip.GzipFile, 'TextIOWrapper']:
        """Open a FASTQ file, handling gzipped and non-gzipped files.
        
        Args:
            filepath: Path to the FASTQ file
            
        Returns:
            An opened file handle
        """
        if filepath.endswith(('.gz', '.gzip')):
            return gzip.open(filepath, "rt")
        else:
            return open(filepath, "rt")
    
    @staticmethod
    def fastq_pair_iterator(r1_path: str, r2_path: str) -> Iterator[Tuple[Tuple, Tuple]]:
        """Create an iterator over paired FASTQ reads.
        
        Args:
            r1_path: Path to the first read file
            r2_path: Path to the second read file
            
        Returns:
            Iterator yielding pairs of (read1, read2) tuples where each read is 
            a tuple of (title, sequence, quality)
        """
        with FastqHandler.open_fastq(r1_path) as f1, FastqHandler.open_fastq(r2_path) as f2:
            # Use the more efficient FastqGeneralIterator
            for read1, read2 in zip(FastqGeneralIterator(f1), FastqGeneralIterator(f2)):
                yield read1, read2
    
    @staticmethod
    def count_fastq_reads(filepath: str) -> int:
        """Count the number of reads in a FASTQ file.
        
        Args:
            filepath: Path to the FASTQ file
            
        Returns:
            Number of reads in the file
        """
        count = 0
        with FastqHandler.open_fastq(filepath) as f:
            for _ in FastqGeneralIterator(f):
                count += 1
        return count

## FASTQ Output Manager

In [ ]:
#| export
class FastqOutputManager:
    """Manages output FASTQ files for different barcode categories."""
    
    def __init__(
        self, 
        output_prefix: str,
        output_dir: str,
        categories: List[str],
        compress: bool = True,
        write_output: bool = True
    ):
        """Initialize the output manager.
        
        Args:
            output_prefix: Prefix for output files
            output_dir: Directory for output files
            categories: List of barcode categories (e.g., "barcode5_orientFR")
            compress: Whether to compress output files
            write_output: Whether to write output files at all
        """
        self.output_prefix = output_prefix
        self.output_dir = output_dir
        self.categories = categories
        self.compress = compress
        self.write_output = write_output
        self.file_handles = {}
        
        # Create output directory if it doesn't exist and we're writing files
        if self.write_output:
            os.makedirs(output_dir, exist_ok=True)
            
            # Initialize file handles for each category
            self._init_file_handles()
    
    def _init_file_handles(self):
        """Initialize file handles for all categories."""
        for category in self.categories:
            self._open_category_files(category)
    
    def _open_category_files(self, category: str):
        """Open R1 and R2 files for a category.
        
        Args:
            category: Barcode category
        """
        r1_path = self._get_output_path(category, "R1")
        r2_path = self._get_output_path(category, "R2")
        
        mode = "wt" if self.compress else "w"
        
        if self.compress:
            self.file_handles[f"{category}_R1"] = gzip.open(r1_path, mode)
            self.file_handles[f"{category}_R2"] = gzip.open(r2_path, mode)
        else:
            self.file_handles[f"{category}_R1"] = open(r1_path, mode)
            self.file_handles[f"{category}_R2"] = open(r2_path, mode)
    
    def _get_output_path(self, category: str, read_suffix: str) -> str:
        """Generate an output file path based on category and read type.
        
        Args:
            category: Barcode category
            read_suffix: R1 or R2
            
        Returns:
            Path to the output file
        """
        if self.compress:
            extension = ".fastq.gz"
        else:
            extension = ".fastq"
            
        filename = f"{self.output_prefix}_{category}_{read_suffix}{extension}"
        return os.path.join(self.output_dir, filename)
    
    def write_read_pair(
        self, 
        category: str, 
        read1: Tuple[str, str, str], 
        read2: Tuple[str, str, str] ):
        """Write a read pair to the appropriate output files.
        
        Args:
            category: Barcode category
            read1: Read 1 tuple (title, sequence, quality)
            read2: Read 2 tuple (title, sequence, quality)
        """
        # Skip writing if not required
        if not self.write_output:
            return
            
        title1, seq1, qual1 = read1
        title2, seq2, qual2 = read2
        
        r1_handle = self.file_handles[f"{category}_R1"]
        r2_handle = self.file_handles[f"{category}_R2"]
        
        # Write in FastqGeneralIterator-compatible format
        r1_handle.write(f"@{title1}\n{seq1}\n+\n{qual1}\n")
        r2_handle.write(f"@{title2}\n{seq2}\n+\n{qual2}\n")
        
        
    def close(self):
        """Close all file handles."""
        for handle in self.file_handles.values():
            handle.close()
        
        self.file_handles = {}
    
    def __enter__(self):
        """Context manager enter method."""
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit method."""
        self.close()

## FASTQ Processing Functions

In [ ]:
#| export
def prepare_fastq_categories(barcodes: List[BarcodeConfig]) -> List[str]:
    """Prepare output categories based on barcodes.
    
    Args:
        barcodes: List of barcode configurations
    
    Returns:
        List of category names
    """
    categories = []
    
    # Determine if we're in single barcode mode or multiple barcode mode
    single_barcode_mode = len(barcodes) == 1 or all(b.location.value == "UNK" for b in barcodes)
    
    if single_barcode_mode:
        categories.extend(["barcode_orientFR", "barcode_orientRC"])
    else:
        # Multiple barcodes with locations
        for barcode in barcodes:
            location = barcode.location.value
            if location in ["5", "3"]:
                categories.extend([
                    f"barcode{location}_orientFR",
                    f"barcode{location}_orientRC"
                ])
    
    # Add no barcode category
    categories.append("noBarcode")
    
    return categories

In [ ]:
#| export
def process_fastq_files(
    config: BarcodeExtractorConfig,
    fastq_files: List[str],
    compress_output: bool = True,
    search_both_reads: bool = True
) -> ExtractionStatistics:
    """Process FASTQ files to extract barcodes.
    
    Args:
        config: Barcode extractor configuration
        fastq_files: List of FASTQ files (either 1 or 2 files)
        compress_output: Whether to compress output files
        search_both_reads: Whether to search for barcodes in both reads of paired-end data
    
    Returns:
        Statistics from the extraction process
    """
    logger = config.logger
    
    # Validate FASTQ files
    if len(fastq_files) not in [1, 2]:
        raise ValueError("Either 1 or 2 FASTQ files must be provided")
    
    # Determine if we have paired-end data
    paired_end = len(fastq_files) == 2
    
    # If a directory is provided, find paired FASTQ files
    if len(fastq_files) == 1 and os.path.isdir(fastq_files[0]):
        r1_path, r2_path = FastqHandler.find_fastq_pairs(fastq_files[0])
        fastq_files = [r1_path, r2_path]
        paired_end = True
    
    logger.info(f"FASTQ files: {fastq_files}")
    
    # Prepare output categories
    categories = prepare_fastq_categories(config.barcodes)
    logger.info(f"Output categories: {categories}")
    
    # Initialize statistics
    stats = ExtractionStatistics()
    
    # Get the total number of reads for progress tracking
    total_reads = FastqHandler.count_fastq_reads(fastq_files[0])
    logger.info(f"Total reads: {total_reads}")
    
    # Process files
    with FastqOutputManager(
        output_prefix=config.output_prefix,
        output_dir=config.output_dir,
        categories=categories,
        compress=compress_output,
        write_output=config.write_output_files  # Pass the flag
    ) as output_manager:
        
        if paired_end:
            # Process paired-end data
            r1_path, r2_path = fastq_files
            read_count = 0
            
            for read1, read2 in tqdm(FastqHandler.fastq_pair_iterator(r1_path, r2_path), 
                                    desc="Processing paired reads", total=total_reads):
                read_count += 1
                stats.total_reads += 1
                
                # Update progress periodically
                if read_count % 100000 == 0 and config.verbose:
                    logger.info(f"Processed {read_count} read pairs")
                
                # Extract title, sequence, and quality
                title1, seq1, qual1 = read1
                title2, seq2, qual2 = read2
                
                # Search for barcodes in read 1
                match1, category1 = classify_read_by_first_match(
                    sequence=seq1,
                    barcodes=config.barcodes,
                    max_mismatches=config.max_mismatches
                )
                
                # If barcode not found in read 1 and we're searching both reads
                if category1 == "noBarcode" and search_both_reads:
                    match2, category2 = classify_read_by_first_match(
                        sequence=seq2,
                        barcodes=config.barcodes,
                        max_mismatches=config.max_mismatches
                    )
                    
                    if category2 != "noBarcode":
                        match = match2
                        category = category2
                    else:
                        match = None
                        category = "noBarcode"
                else:
                    match = match1
                    category = category1
                
                # Update statistics
                if match:
                    stats.update_barcode_match(match)
                
                # Write to appropriate output file
                output_manager.write_read_pair(category, read1, read2)
            
            logger.info(f"Finished processing {read_count} read pairs")
        else:
            # Process single-end data (duplicated for paired output compatibility)
            logger.warning("Processing single FASTQ file as paired (duplicating reads)")
            r1_path = fastq_files[0]
            
            read_count = 0
            
            with FastqHandler.open_fastq(r1_path) as f1:
                for title, seq, qual in tqdm(FastqGeneralIterator(f1), 
                                           desc="Processing single reads", total=total_reads):
                    read_count += 1
                    stats.total_reads += 1
                    
                    # Update progress periodically
                    if read_count % 100000 == 0 and config.verbose:
                        logger.info(f"Processed {read_count} reads")
                    
                    # Package as a read tuple
                    read = (title, seq, qual)
                    
                    # Search for barcodes
                    match, category = classify_read_by_first_match(
                        sequence=seq,
                        barcodes=config.barcodes,
                        max_mismatches=config.max_mismatches
                    )
                    
                    # Update statistics
                    if match:
                        stats.update_barcode_match(match)
                    
                    # Write to appropriate output file (duplicating the read for both R1 and R2)
                    output_manager.write_read_pair(category, read, read)
            
            logger.info(f"Finished processing {read_count} reads")
    
    # Save statistics
    # When not writing sequence files, create output directory if it doesn't exist
    if not config.write_output_files:
        os.makedirs(config.output_dir, exist_ok=True)
    save_statistics(stats, config.output_prefix, config.output_dir)
    
    return stats

In [ ]:
#| export
def save_statistics(stats: ExtractionStatistics, output_prefix: str, output_dir: str) -> None:
    """Save extraction statistics to files.
    
    Args:
        stats: Extraction statistics
        output_prefix: Prefix for output files
        output_dir: Directory for output files
    """
    # Save as JSON
    json_path = os.path.join(output_dir, f"{output_prefix}_extraction_stats.json")
    stats.save_json(json_path)
    
    # Save as TSV
    tsv_path = os.path.join(output_dir, f"{output_prefix}_extraction_stats.tsv")
    stats.save_tsv(tsv_path)

In [ ]:
# Example usage with test FASTQ files
from BarcodeSeqKit.core import BarcodeConfig, BarcodeLocationType, BarcodeExtractorConfig

# Define barcodes to search for
barcodes = [
    BarcodeConfig(
        sequence="TAACTGAGGCCGGC",  # 3' barcode 
        location=BarcodeLocationType.THREE_PRIME,
        name="3prime",
        description="3' barcode from test data"
    ),
    BarcodeConfig(
        sequence="CTGACTCCTTAAGGGCC",  # 5' barcode
        location=BarcodeLocationType.FIVE_PRIME,
        name="5prime",
        description="5' barcode from test data"
    )
]

# Create a configuration
output_dir = "../tests/fastq_output"
os.makedirs(output_dir, exist_ok=True)

config = BarcodeExtractorConfig(
    barcodes=barcodes,
    output_prefix="test_extraction",
    output_dir=output_dir,
    max_mismatches=0,
    verbose=True
)

# Path to test FASTQ files
fastq_dir = "../tests"
test_fastq1 = os.path.join(fastq_dir, "test.1.fastq.gz")
test_fastq2 = os.path.join(fastq_dir, "test.2.fastq.gz")

# Check if test files exist
if os.path.exists(test_fastq1) and os.path.exists(test_fastq2):
    print(f"Processing FASTQ files: {test_fastq1}, {test_fastq2}")
    
    # Process the FASTQ files
    stats = process_fastq_files(
        config=config,
        fastq_files=[test_fastq1, test_fastq2],
        compress_output=True,
        search_both_reads=True
    )
    
    # Print results
    print(f"\nTotal reads: {stats.total_reads}")
    print(f"Total barcode matches: {stats.total_barcode_matches}")
    
    for barcode_name, count in stats.matches_by_barcode.items():
        print(f"  {barcode_name}: {count} matches")
    
    for orientation, count in stats.matches_by_orientation.items():
        print(f"  Orientation {orientation}: {count} matches")
    
    for category, count in stats.matches_by_category.items():
        print(f"  Category {category}: {count} matches")
    
    # List the output files
    output_files = [f for f in os.listdir(output_dir) if f.startswith("test_extraction_") and f.endswith((".fastq.gz", ".fastq"))]
    print("\nOutput files:")
    for f in output_files:
        path = os.path.join(output_dir, f)
        if f.endswith(".fastq.gz"):
            try:
                # Count FASTQ reads
                read_count = FastqHandler.count_fastq_reads(path)
                print(f"  {f} ({read_count} reads)")
            except Exception as e:
                # Fallback to file size
                size = os.path.getsize(path)
                print(f"  {f} ({size} bytes) - Error getting read count: {str(e)}")
        else:
            size = os.path.getsize(path)
            print(f"  {f} ({size} bytes)")
else:
    print(f"Test files not found. Expected: {test_fastq1} and {test_fastq2}")

2025-03-24 13:58:40,722 - BarcodeSeqKit - INFO - FASTQ files: ['../tests/test.1.fastq.gz', '../tests/test.2.fastq.gz']
2025-03-24 13:58:40,723 - BarcodeSeqKit - INFO - Output categories: ['barcode3_orientFR', 'barcode3_orientRC', 'barcode5_orientFR', 'barcode5_orientRC', 'noBarcode']
2025-03-24 13:58:40,727 - BarcodeSeqKit - INFO - Total reads: 247


Processing FASTQ files: ../tests/test.1.fastq.gz, ../tests/test.2.fastq.gz


Processing paired reads:   0%|          | 0/247 [00:00<?, ?it/s]

2025-03-24 13:58:40,787 - BarcodeSeqKit - INFO - Finished processing 247 read pairs



Total reads: 247
Total barcode matches: 18
  5prime: 10 matches
  3prime: 8 matches
  Orientation FR: 18 matches
  Category barcode5_orientFR: 10 matches
  Category barcode3_orientFR: 8 matches

Output files:
  test_extraction_barcode3_orientRC_R2.fastq.gz (0 reads)
  test_extraction_noBarcode_R2.fastq.gz (229 reads)
  test_extraction_barcode3_orientFR_R1.fastq.gz (8 reads)
  test_extraction_barcode5_orientRC_R1.fastq.gz (0 reads)
  test_extraction_barcode5_orientFR_R2.fastq.gz (10 reads)
  test_extraction_noBarcode_R1.fastq.gz (229 reads)
  test_extraction_barcode3_orientFR_R2.fastq.gz (8 reads)
  test_extraction_barcode3_orientRC_R1.fastq.gz (0 reads)
  test_extraction_barcode5_orientFR_R1.fastq.gz (10 reads)
  test_extraction_barcode5_orientRC_R2.fastq.gz (0 reads)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()